In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
import numpy as np
from google.colab import drive

# توصيل Google Drive
drive.mount('/content/drive')

input_texts = ['hi', 'hello', 'thanks', 'yes', 'no']
target_texts = ['مرحبا', 'اهلا', 'شكرا', 'نعم', 'لا']

input_characters = sorted(set(''.join(input_texts)))
target_characters = sorted(set(''.join(target_texts)))

num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

input_token_index = {char: i for i, char in enumerate(input_characters)}
target_token_index = {char: i for i, char in enumerate(target_characters)}
reverse_target_char_index = {i: char for char, i in target_token_index.items()}

encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype='float32')
decoder_input_data = np.zeros((len(target_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')
decoder_target_data = np.zeros((len(target_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')

for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0

latent_dim = 256

encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='categorical_crossentropy')

model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=64, epochs=100)

model_save_path = '/content/drive/MyDrive/simple_seq2seq_model.h5'
model.save(model_save_path)
print(f"Model saved at {model_save_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Epoch 1/100
1/1 [==============================] - 4s 4s/step - loss: 1.2461
Epoch 2/100
1/1 [==============================] - 0s 31ms/step - loss: 1.2356
Epoch 3/100
1/1 [==============================] - 0s 32ms/step - loss: 1.2249
Epoch 4/100
1/1 [==============================] - 0s 35ms/step - loss: 1.2138
Epoch 5/100
1/1 [==============================] - 0s 36ms/step - loss: 1.2021
Epoch 6/100
1/1 [==============================] - 0s 30ms/step - loss: 1.1895
Epoch 7/100
1/1 [==============================] - 0s 30ms/step - loss: 1.1754
Epoch 8/100
1/1 [==============================] - 0s 30ms/step - loss: 1.1595
Epoch 9/100
1/1 [==============================] - 0s 29ms/step - loss: 1.1409
Epoch 10/100
1/1 [==============================] - 0s 34ms/step - loss: 1.1188
Epoch 11/100
1/1 [==============================] - 0s 36ms/step - loss: 1.0921
Ep